# DeepLIFT on Tox21 dataset for GCN

In [1]:
import os
import os.path as osp

import torch
from torch.utils.data import random_split
from torch_geometric.data import download_url, extract_zip
from torch_geometric.loader import DataLoader

from dig.xgraph.dataset import MoleculeDataset
from dig.xgraph.models import GCN_3l
from dig.xgraph.utils.compatibility import compatible_state_dict
from dig.xgraph.utils.init import fix_random_seed

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/data/shurui.gui/anaconda3/envs/torch_110/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load dataset

In [2]:
def split_dataset(dataset, dataset_split=(0.8, 0.1, 0.1)):
    dataset_len = len(dataset)
    dataset_split = [int(dataset_len * dataset_split[0]),
                     int(dataset_len * dataset_split[1]),
                     0]
    dataset_split[2] = dataset_len - dataset_split[0] - dataset_split[1]
    train_set, val_set, test_set = random_split(dataset, dataset_split)

    return {'train': train_set, 'val': val_set, 'test': test_set}


fix_random_seed(123)
dataset = MoleculeDataset('datasets', 'Tox21')
dataset.data.x = dataset.data.x.to(torch.float32)
dataset.data.y = dataset.data.y[:, 2]  # the target 2 task.
dim_node = dataset.num_node_features
dim_edge = dataset.num_edge_features
num_targets = dataset.num_classes
num_classes = 2

splitted_dataset = split_dataset(dataset)
dataloader = DataLoader(splitted_dataset['test'], batch_size=1, shuffle=False)

### Load model and checkpoints

In [3]:
def check_checkpoints(root='./'):
    if osp.exists(osp.join(root, 'checkpoints')):
        return
    url = ('https://github.com/divelab/DIG_storage/raw/main/xgraph/checkpoints.zip')
    path = download_url(url, root)
    extract_zip(path, root)
    os.unlink(path)


model = GCN_3l(model_level='graph', dim_node=dim_node, dim_hidden=300, num_classes=num_classes)
model.to(device)
check_checkpoints()
ckpt_path = osp.join('checkpoints', 'tox21', 'GCN_3l', '2', 'GCN_3l_best.ckpt')
state_dict = compatible_state_dict(torch.load(ckpt_path, map_location='cpu')['state_dict'])
model.load_state_dict(state_dict)

<All keys matched successfully>

### Display example output

In [4]:
data = list(dataloader)[0].to(device)
out = model(data.x, data.edge_index)
print(out)

tensor([[ 1.9660, -1.8921]], device='cuda:3', grad_fn=<AddmmBackward0>)


### Load the explainer

In [5]:
from dig.xgraph.method import DeepLIFT

explainer = DeepLIFT(model, explain_graph=True)

### Setup for evaluation

In [6]:
# --- Set the Sparsity to 0.5 ---
sparsity = 0.5

# --- Create data collector and explanation processor ---
from dig.xgraph.evaluation import XCollector

x_collector = XCollector(sparsity)
# x_processor = ExplanationProcessor(model=model, device=device)

### Run explainer on the given model and dataset

In [7]:
for index, data in enumerate(dataloader):
    print(f'explain graph line {dataloader.dataset.indices[index] + 2}')
    data.to(device)

    if torch.isnan(data.y[0].squeeze()):
        continue

    walks, masks, related_preds = explainer(data.x, data.edge_index, sparsity=sparsity, num_classes=num_classes)

    x_collector.collect_data(masks, related_preds, data.y[0].squeeze().long().item())

    # if you only have the edge masks without related_pred, please feed sparsity controlled mask to
    # obtain the result: x_processor(data, masks, x_collector)

    if index >= 99:
        break

explain graph line 3421
explain graph line 2536
explain graph line 7266
explain graph line 2650
explain graph line 793
explain graph line 7708
explain graph line 5171
explain graph line 5670
explain graph line 1400
explain graph line 655
explain graph line 2746
explain graph line 5111
explain graph line 3959
explain graph line 5602
explain graph line 2661
explain graph line 1500
explain graph line 3117
explain graph line 1056
explain graph line 6898
explain graph line 2455
explain graph line 4479
explain graph line 408
explain graph line 892
explain graph line 5199
explain graph line 7001
explain graph line 2840
explain graph line 5358
explain graph line 3284
explain graph line 1390
explain graph line 226
explain graph line 7308
explain graph line 6204
explain graph line 194
explain graph line 2501
explain graph line 6398
explain graph line 1191
explain graph line 529
explain graph line 6630
explain graph line 7686
explain graph line 6699
explain graph line 969
explain graph line 6030


### Output metrics evaluation results

In [8]:
print(f'Fidelity: {x_collector.fidelity:.4f}\n'
      f'Fidelity_inv: {x_collector.fidelity_inv:.4f}\n'
      f'Sparsity: {x_collector.sparsity:.4f}')

Fidelity: 0.1517
Fidelity_inv: -0.0718
Sparsity: 0.5000
